In [1]:
import pandas as pd

In [2]:
drinks = pd.read_csv('http://bit.ly/drinksbycountry')
drinks.head()

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Afghanistan,0,0,0,0.0,Asia
1,Albania,89,132,54,4.9,Europe
2,Algeria,25,0,14,0.7,Africa
3,Andorra,245,138,312,12.4,Europe
4,Angola,217,57,45,5.9,Africa


In [3]:
drinks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 6 columns):
country                         193 non-null object
beer_servings                   193 non-null int64
spirit_servings                 193 non-null int64
wine_servings                   193 non-null int64
total_litres_of_pure_alcohol    193 non-null float64
continent                       193 non-null object
dtypes: float64(1), int64(3), object(2)
memory usage: 9.1+ KB


จากข้อมูลที่ดูผ่าน info() พบว่ามีการใช้งานหน่วยความจำอย่างน้อย 9.1+ KB เนื่องจากตัว info() ไม่สามารถนับการใช้งานจริงของตอลัมน์ที่มีชนิดของข้อมูลเป็น object ได้ตรงๆ แต่ถ้าต้องการดูค่าที่แท้จริงแล้วให้ใส่พารามิเตอร์ memory_usage='deep' กำกับดังนี้

In [4]:
drinks.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 6 columns):
country                         193 non-null object
beer_servings                   193 non-null int64
spirit_servings                 193 non-null int64
wine_servings                   193 non-null int64
total_litres_of_pure_alcohol    193 non-null float64
continent                       193 non-null object
dtypes: float64(1), int64(3), object(2)
memory usage: 30.4 KB


แล้วถ้าเราต้องการดูการใช้งานหน่วยความจำของแต่ละคอลัมน์ได้ดังนี้

In [5]:
drinks.memory_usage()

Index                             80
country                         1544
beer_servings                   1544
spirit_servings                 1544
wine_servings                   1544
total_litres_of_pure_alcohol    1544
continent                       1544
dtype: int64

จากด้านบนจะเห็นว่ายังไม่ได้แสดงค่าการใช้งานที่แท้จริง เราต้องส่งพารามิเตอร์ deep=True เข้าไปด้วยดังนี้ ปล.หน่วยเป็นไบต์

In [6]:
drinks.memory_usage(deep=True)

Index                              80
country                         12588
beer_servings                    1544
spirit_servings                  1544
wine_servings                    1544
total_litres_of_pure_alcohol     1544
continent                       12332
dtype: int64

In [7]:
drinks.memory_usage(deep=True).sum()

31176

จะเห็นว่าคอลัมน์ที่มีการใช้งานหน่วยความจำมากจะเป็นคอลัมน์ที่มีชนิดของข้อมูลเป็น object แล้วเรามีวิธีการใดบ้างล่ะที่จะประหยัดการใช้ังานหน่วยความจำได้บ้าง เช่นเราอาจเปลี่ยนวิธีการจัดเก็บจาก ชุดข้อความ เป็น ตัวเลข เริ่มจากลองสำรวจคอลัมน์ continent ดูก่อนว่ามีชุดข้อความที่มีความแตกต่างกันเป็นจำนวนเท่าเท่าใด

In [8]:
sorted(drinks.continent.unique())

['Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South America']

จะพบว่าในคอลัมน์ continent นั้นมีค่าที่แตกต่างกันอยู่ 6 ค่า ดังนั้นเราอาจแทนที่ข้อมูลชุดตัวอักษรด้วยตัวเลข 0-5 ด้วย lookup table ได้ดังนี้

In [9]:
drinks.continent = drinks.continent.astype('category')
drinks.dtypes

country                           object
beer_servings                      int64
spirit_servings                    int64
wine_servings                      int64
total_litres_of_pure_alcohol     float64
continent                       category
dtype: object

In [10]:
drinks.continent.head()

0      Asia
1    Europe
2    Africa
3    Europe
4    Africa
Name: continent, dtype: category
Categories (6, object): [Africa, Asia, Europe, North America, Oceania, South America]

In [11]:
drinks.continent.cat.codes.head()

0    1
1    2
2    0
3    2
4    0
dtype: int8

In [12]:
drinks.memory_usage(deep=True)

Index                              80
country                         12588
beer_servings                    1544
spirit_servings                  1544
wine_servings                    1544
total_litres_of_pure_alcohol     1544
continent                         744
dtype: int64

จะพบว่าในคอลัมน์ continent นั้นมีการใช้งานหน่วยความจำลดลงมาก

ต่อมาเราทดลองสร้าง pandas DataFrame ขึ้นมาทดสอบ 1 ชุดดังนี้

In [13]:
df = pd.DataFrame({'ID':[100, 101, 102, 103], 'quality':['good', 'very good', 'good', 'excelent']})
df

,ID,quality
0,100,good
1,101,very good
2,102,good
3,103,excelent


In [14]:
df.sort_values('quality')

,ID,quality
3,103,excelent
0,100,good
2,102,good
1,101,very good


จากการเรียงลำดับคอลัมน์ quality พบว่าจริงๆแล้วเราคาดหวังว่าการเรียงลำดับต้องเป็น good --> very good --> excelent แต่ในที่นี้เป็นการเรียงลำดับด้วยตัวอักษร แต่เราสามารถปรับวิธีการเรียงลำดับของคอลัมน์ quality ได้ดังนี้

In [15]:
df.quality = df.quality.astype('category', categories=['good', 'very good', 'excelent'], ordered=True)

In [16]:
df.quality

0         good
1    very good
2         good
3     excelent
Name: quality, dtype: category
Categories (3, object): [good < very good < excelent]

In [17]:
df.sort_values('quality')

,ID,quality
0,100,good
2,102,good
1,101,very good
3,103,excelent


เรายังสามารถทำการเปรียบเทียบในคอลัมน์ quality ได้ดังนี้

In [18]:
df.loc[df.quality > 'good', :]

,ID,quality
1,101,very good
3,103,excelent
